In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
annotations_path = []
for dirname, _, filenames in os.walk('/kaggle/input/face-mask-detection/annotations'):
    for filename in filenames:
        annotations_path.append(os.path.join(dirname, filename))
images_path = []
for dirname, _, filenames in os.walk('/kaggle/input/face-mask-detection/images'):
    for filename in filenames:
        images_path.append(os.path.join(dirname, filename))
        
annotations_path.sort()
images_path.sort()

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Létrehozunk egy map-et, melyben minden képhez fog tartozni egy "BoundingBox" tömb
import xml.etree.ElementTree as et

class BoundingBox :
    def __init__(self, xmin, ymin, xmax, ymax, label):
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax
        self.label = label
        
class ImageSize:
     def __init__(self, x, y):
        self.x = x
        self.y = y

image_boxes_assoc = {}
image_size_assoc = {}

for annotation_path in annotations_path:
    with open(annotation_path) as annotation:
        xml = et.parse(annotation)
        root = xml.getroot()
        img = root[1].text
        image_size_assoc[img] = ImageSize(root[2][0].text,root[2][1].text )
        
        bounding_boxes = []
        for i in range(4,len(root)):
            if((int(root[i][5][2].text)-int(root[i][5][0].text))>10 and (int(root[i][5][3].text)-int(root[i][5][1].text))>10):
                bounding_boxes.append(BoundingBox(root[i][5][0].text,
                                           root[i][5][1].text, root[i][5][2].text, root[i][5][3].text,root[i][0].text))
        #for i in bounding_boxes:
        #    print(img, i.xmin, " ", i.ymin, " ", i.xmax, " ", i.ymax, " ", i.label )
        image_boxes_assoc[img] = bounding_boxes

In [ ]:
#Létrehozzuk a dobozokat

import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing.image import img_to_array
import random as random

images = []
labels = []

image_size = 150

for img_path in images_path:
    img_name = img_path.split('images/')[1]
    img = cv2.imread(img_path)
    cv2.resize(img, (int(image_size_assoc[img_name].x),
                     int(image_size_assoc[img_name].y)))
    
    for box in image_boxes_assoc[img_name]:
        if box.label== 'with_mask':
            #if(random.random()<0.75):
            #    continue
            labels.append(0)
        elif box.label=='without_mask':
            labels.append(1)
        else:
            labels.append(2)
        face = img[int(box.ymin) : int(box.ymax), int(box.xmin) : int(box.xmax)]
        face = cv2.resize(face, (image_size, image_size))
        face = img_to_array(face)
        images.append(face)
        
a = np.array(labels)
unique, counts = np.unique(a, return_counts=True)
dict(zip(unique, counts))

In [ ]:
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

train_label_Ids_keras = np_utils.to_categorical(labels, num_classes=3)
x_train, x_test, y_train, y_test = train_test_split(
    images, train_label_Ids_keras, test_size = 0.1, shuffle=True)

train_original_size = len(x_train)
for i in range(train_original_size):
    if y_train[i][1]==1:
        for j in range(5):
            x_train.append(x_train[i])
            without_mask_label =  np.array([[0, 1, 0]])
            y_train = np.concatenate((y_train, without_mask_label))
    elif y_train[i][2]==1:
        for j in range(20):
            x_train.append(x_train[i])
            incorrect_mask_label =  np.array([[0, 0, 1]])
            y_train = np.concatenate((y_train, incorrect_mask_label))
        

x_train = np.array(x_train)
x_test = np.array(x_test)

BATCH_SIZE = 16


train_gen = ImageDataGenerator(
        zoom_range=0.2,
        rotation_range=15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=preprocess_input)

train_generator = train_gen.flow(
    np.array(x_train),
    y_train,
    batch_size = BATCH_SIZE,
    shuffle = True
)
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow(
    np.array(x_test),
    y_test,
    batch_size = BATCH_SIZE,
    shuffle = False)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization

'''
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(Conv2D(32, (5, 5), activation='relu'))
model.add(MaxPool2D((2, 2)))
          
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
          
model.add(Conv2D(128, kernel_size=3, activation = 'relu'))
          
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))
model.summary() 

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

model.summary()

'''

In [ ]:

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model


model = Sequential()
base_model = VGG16(weights='imagenet', include_top=False,input_shape=(image_size,image_size,3))
for layer in base_model.layers[:-4]:
    layer.trainable=False
      
model.add(base_model)
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))

model.summary()


In [ ]:

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
'''
model = Sequential()
base_model = InceptionV3(weights='imagenet', include_top=False,input_shape=(image_size,image_size,3))
for layer in base_model.layers[:-16]:
    layer.trainable=False
      
model.add(base_model)
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='softmax'))

model.summary()
'''

In [ ]:
from tensorflow.keras.optimizers import Adam
adam = Adam(lr=1e-5)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

history = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=20,
        validation_data=test_generator) 

print('\nhistory dict:', history.history)

In [ ]:
import matplotlib.pyplot as plt

train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
train_loss = history.history['loss']
val_loss = history.history['val_loss']
No_Of_Epochs = range(20)

plt.plot(No_Of_Epochs, train_acc, marker = 'o', color = 'blue', markersize = 12, 
                 linewidth = 2, label = 'Training Accuracy')
plt.plot(No_Of_Epochs, val_acc, marker = '.', color = 'red', markersize = 12, 
                 linewidth = 2, label = 'Validation Accuracy')

plt.title('Training Accuracy and Testing Accuracy w.r.t Number of Epochs')

plt.legend()

plt.figure()

plt.plot(No_Of_Epochs, train_loss, marker = 'o', color = 'blue', markersize = 12, 
                 linewidth = 2, label = 'Training Loss')
plt.plot(No_Of_Epochs, val_loss, marker = '.', color = 'red', markersize = 12, 
                 linewidth = 2, label = 'Validation Loss')

plt.title('Training Loss and Testing Loss w.r.t Number of Epochs')

plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import classification_report

x_test = [i / 255 for i in x_test]


Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(np.array(x_test))
print(classification_report(Y_test, y_pred))

In [ ]:
from keras.models import model_from_json

#serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")